# OpenIE

In [ ]:
!pip install stanza

# Import stanza
import stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 28.5 MB/s eta 0:00:00


In [ ]:
corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

INFO:stanza:Installing CoreNLP package into ./corenlp


INFO:stanza:Downloaded file to ./corenlp/corenlp.zip


In [ ]:
!ls $CORENLP_HOME

build.xml				  LIBRARY-LICENSES
corenlp.sh				  LICENSE.txt
CoreNLP-to-HTML.xsl			  Makefile
ejml-core-0.39.jar			  patterns
ejml-core-0.39-sources.jar		  pom-java-11.xml
ejml-ddense-0.39.jar			  pom-java-17.xml
ejml-ddense-0.39-sources.jar		  pom.xml
ejml-simple-0.39.jar			  protobuf-java-3.19.6.jar
ejml-simple-0.39-sources.jar		  README.txt
input.txt				  RESOURCE-LICENSES
input.txt.out				  sample-project-pom.xml
input.txt.xml				  SemgrexDemo.java
istack-commons-runtime-3.0.7.jar	  ShiftReduceDemo.java
istack-commons-runtime-3.0.7-sources.jar  slf4j-api.jar
javax.activation-api-1.2.0.jar		  slf4j-simple.jar
javax.activation-api-1.2.0-sources.jar	  stanford-corenlp-4.5.7.jar
javax.json-api-1.0-sources.jar		  stanford-corenlp-4.5.7-javadoc.jar
javax.json.jar				  stanford-corenlp-4.5.7-models.jar
jaxb-api-2.4.0-b180830.0359.jar		  stanford-corenlp-4.5.7-sources.jar
jaxb-api-2.4.0-b180830.0359-sources.jar   StanfordCoreNlpDemo.java
jaxb-impl-2.4.0-b180830.0438.jar	  StanfordDependenci

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Import client module
from stanza.server import CoreNLPClient

In [ ]:
# Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
client = CoreNLPClient(
    annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner','openie'],
    memory='4G',
    endpoint='http://localhost:9001',
    max_char_length = 130000,
    timeout = 50000,
    be_quiet=False)
print(client)

# Start the background server and wait for some time
# Note that in practice this is totally optional, as by default the server will be started when the first annotation is performed
client.start()
import time; time.sleep(10)

INFO:stanza:Writing properties to tmp file: corenlp_server-cfa688bcf69f4a63.props
INFO:stanza:Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 50000 -threads 5 -maxCharLength 130000 -quiet False -serverProperties corenlp_server-cfa688bcf69f4a63.props -annotators tokenize,ssplit,pos,lemma,ner,openie -preload -outputFormat serialized


In [ ]:
#text = open("/content/drive/MyDrive/01-Research/Geo-Isa/Test/Input_all.txt", "r")
text = open("/content/drive/MyDrive/01-Research/Geo-Isa/Coding/Evaluation_1600 tweets/evaluation-1600-annotated-tweets.txt", "r")
data = text.readlines()

In [ ]:
def process_text_chunk(text_chunk):
    # Ensure text_chunk is a string
    document = client.annotate(text_chunk, output_format='json')
    triples_chunk = []
    for sentence in document['sentences']:
        for triple in sentence['openie']:
            triples_chunk.append({
                'subject': triple['subject'],
                'relation': triple['relation'],
                'object': triple['object'],
            })
    return triples_chunk

max_length = 50000 ## change if run into timeout issues
triples = []

current_chunk = ""
for line in data:
    if len(current_chunk) + len(line) > max_length:
        # Process the current chunk since adding the next line will exceed the limit
        triples.extend(process_text_chunk(current_chunk))
        current_chunk = line  # Start a new chunk with the current line
    else:
        current_chunk += line  # Add the current line to the chunk

# process the last chunk with size less than 130000
if current_chunk:
    triples.extend(process_text_chunk(current_chunk))

NameError: name 'data' is not defined

In [ ]:
import csv
with open('GeoIsa_OpenIE_evaluation_1600_triples.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['subject', 'relation', 'object'])
    writer.writeheader()
    writer.writerows(triples)

In [ ]:
##STOP HERE

In [ ]:
### 9/12/24 for evaluation set (1600) for GeoISA
def process_text_chunk(text_chunk):
    # Ensure text_chunk is a string
    document = client.annotate(text_chunk, output_format='json')
    triples_chunk = []
    for sentence in document['sentences']:
        for triple in sentence['openie']:
            triples_chunk.append({
                'subject': triple['subject'],
                'relation': triple['relation'],
                'object': triple['object'],
            })
    return triples_chunk

max_length = 50000  # change if run into timeout issues
triples = []

current_chunk = ""
with open('/content/drive/MyDrive/01-Research/Geo-Isa/Coding/evaluation-1600-annotated-tweets.txt', 'r') as file:
    for line in file:
        if len(current_chunk) + len(line) > max_length:
            # Process the current chunk since adding the next line will exceed the limit
            triples.extend(process_text_chunk(current_chunk))
            current_chunk = line  # Start a new chunk with the current line
        else:
            current_chunk += line  # Add the current line to the chunk

    # Process the last chunk with size less than max_length
    if current_chunk:
        triples.extend(process_text_chunk(current_chunk))

# Define CSV file path
csv_file_path = '/content/drive/MyDrive/01-Research/Geo-Isa/Coding/OpenIE_evaluation_1600_annotated_tweets.csv'

# Write the triples to a CSV file
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.DictWriter(csvfile, fieldnames=['subject', 'relation', 'object'])
    csv_writer.writeheader()
    csv_writer.writerows(triples)

print(f"Triples saved as CSV to {csv_file_path}")

Triples saved as CSV to /content/drive/MyDrive/01-Research/Geo-Isa/Coding/OpenIE_evaluation_1600_annotated_tweets.csv


## This is for the coded triples

In [ ]:
#text = open("/content/drive/MyDrive/01-Research/Geo-Isa/Test/heuristics-150-input.txt", "r")
text = open("/content/drive/MyDrive/01-Research/Geo-Isa/Coding/Evaluation_1600 tweets/evaluation-1600-annotated-tweets.txt", "r")
data = text.readlines()

In [ ]:
import csv

# Process each line separately and extract triples
triples_with_sentences = []

for line in data:
    document = client.annotate(line, output_format='json')

    for sentence in document['sentences']:
        # Reconstruct sentence text from tokens
        sentence_text = ' '.join([token['word'] for token in sentence['tokens']])

        for triple in sentence['openie']:
            triple_data = {
                'sentence': sentence_text,
                'subject': triple['subject'],
                'relation': triple['relation'],
                'object': triple['object'],
            }
            triples_with_sentences.append(triple_data)

# Write triples and sentences to a CSV file
with open('OpenIE_coded_triples.csv', 'w', newline='', encoding='utf-8') as file:
    fieldnames = ['sentence', 'subject', 'relation', 'object']
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()
    for triple in triples_with_sentences:
        writer.writerow(triple)

In [ ]:
## Produce coded triples for evaluation set
# Read tweets from file

with open('/content/drive/MyDrive/01-Research/Geo-Isa/Coding/Evaluation_1600 tweets/evaluation-1600-annotated-tweets.txt', 'r', encoding='utf-8') as file:
    tweets = file.readlines()

tweets = tweets[:1672]

# Process each tweet separately and extract one triple per tweet or use 'None' values
triples_with_sentences = []

for i, tweet in enumerate(tweets, 1):
    # Remove newline characters
    tweet = tweet.strip()

    print(f"Processing tweet {i} of {len(tweets)}")

    try:
        # Annotate the tweet using the pre-initialized client
        annotations = client.annotate(tweet)

        # Initialize triple_data with 'None' values
        triple_data = {
            'sentence': tweet,  # Use the original tweet as the sentence
            'subject': 'None',
            'relation': 'None',
            'object': 'None'
        }

        # Process the first sentence in the tweet
        if annotations.sentence:
            sentence = annotations.sentence[0]

            # Extract the first triple if available
            if sentence.openieTriple:
                triple = sentence.openieTriple[0]
                triple_data.update({
                    'subject': triple.subject,
                    'relation': triple.relation,
                    'object': triple.object,
                })
            else:
                print(f"No triple found for tweet {i}")
        else:
            print(f"No sentence found for tweet {i}")

        triples_with_sentences.append(triple_data)

    except Exception as e:
        print(f"Error processing tweet {i}: {str(e)}")
        # Even in case of error, add an entry with 'None' values
        triples_with_sentences.append({
            'sentence': tweet,
            'subject': 'None',
            'relation': 'None',
            'object': 'None'
        })

# Write triples and sentences to a CSV file
try:
    with open('OpenIE_coded_triples.csv', 'w', newline='', encoding='utf-8') as file:
        fieldnames = ['sentence', 'subject', 'relation', 'object']
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        writer.writeheader()
        for triple in triples_with_sentences:
            writer.writerow(triple)
except IOError:
    print("Error: Could not write to 'OpenIE_coded_triples.csv'.")
    exit(1)

print(f"Processed {len(tweets)} tweets and extracted {len(triples_with_sentences)} entries.")
print("Results saved in 'OpenIE_coded_triples.csv'.")

Processing tweet 1 of 1672
Processing tweet 2 of 1672
Processing tweet 3 of 1672
Processing tweet 4 of 1672
Processing tweet 5 of 1672
Processing tweet 6 of 1672
Processing tweet 7 of 1672
Processing tweet 8 of 1672
Processing tweet 9 of 1672
No triple found for tweet 9
Processing tweet 10 of 1672
Processing tweet 11 of 1672
Processing tweet 12 of 1672
No triple found for tweet 12
Processing tweet 13 of 1672
No triple found for tweet 13
Processing tweet 14 of 1672
No triple found for tweet 14
Processing tweet 15 of 1672
Processing tweet 16 of 1672
Processing tweet 17 of 1672
Processing tweet 18 of 1672
Processing tweet 19 of 1672
Processing tweet 20 of 1672
Processing tweet 21 of 1672
Processing tweet 22 of 1672
Processing tweet 23 of 1672
Processing tweet 24 of 1672
Processing tweet 25 of 1672
Processing tweet 26 of 1672
No triple found for tweet 26
Processing tweet 27 of 1672
Processing tweet 28 of 1672
Processing tweet 29 of 1672
Processing tweet 30 of 1672
Processing tweet 31 of 16